# library loading

In [5]:
#!pip install ko-sentence-transformers 
from sentence_transformers import SentenceTransformer, models
from ko_sentence_transformers.models import KoBertTransformer

# Model loading
word_embedding_model = KoBertTransformer("monologg/kobert", max_seq_length=75)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

from sentence_transformers import SentenceTransformer, util
import numpy as np  
import pandas as pd

# embedder loading
embedder = SentenceTransformer("jhgan/ko-sbert-sts")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


# data loading

In [6]:
# 데이터 불러오기
df = pd.read_csv('DBdata.csv')  # 경로 수정

# 기본 수행

In [7]:
# 전체 카테고리(tot)
## corpus
texts = ''
for line in df.rv[:]:
    texts = texts + line
text = list(filter(None, texts.split(sep='\n')))
## embedding
tot_corpus = text
tot_embedding = embedder.encode(tot_corpus, convert_to_tensor = True)

In [8]:
# 기초케어 카테고리(bas)
## 범위 split
fir = df[df['cate'] == '토너'].index[0]
las = df[df['cate'] == '미스트'].index[-1]
## corpus
texts = ''
for line in df.rv[fir:las]:
    texts = texts + line
text = list(filter(None, texts.split(sep='\n')))

## embedding
bas_corpus = text
bas_embedding = embedder.encode(bas_corpus, convert_to_tensor = True)

In [9]:
# 색조케어 카테고리(col)
## 범위 split
fir = df[df['cate'] == '립'].index[0]
las = df[df['cate'] == '아이'].index[-1]
## corpus
texts = ''
for line in df.rv[fir:las]:
    texts = texts + line
text = list(filter(None, texts.split(sep='\n')))

## embedding
col_corpus = text
col_embedding = embedder.encode(col_corpus, convert_to_tensor = True)

# 필요한 작업 수행

In [23]:
# 출력폼 함수
def menu():
    print('-'*30)
    print('★원하는 카테고리를 선택해주세요★')
    print('1. 전체')
    print('2. 기초')
    print('3. 색조')
    print('-'*30)
    embed, corpus, cate = choice_cate()
    return embed, corpus, cate 
    
    
# 카테고리 선택 함수
def choice_cate():
    num = int(input('→ '))
    if num == 1:
        embed = tot_embedding
        corpus = tot_corpus
        cate = '전체'
    elif num == 2:
        embed = bas_embedding
        corpus = bas_corpus
        cate = '기초'
    elif num == 3:
        embed = col_embedding
        corpus = col_corpus
        cate = '색조'
    else:
        print('잘못된 선택입니다. 1~3 사이의 숫자로 입력해주세요.')
        return menu()
    return embed, corpus, cate
        

# 유사 문장 리턴함수
def num_return(query, embed, corpus, cate):

    cate_embedding = embed
    # 유사도 평가를 위한 작업
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, cate_embedding)[0]
    cos_scores = cos_scores.cpu()
    # 상위 결과 추출
    top_results = np.argpartition(-cos_scores, range(5))[0:5]
    print("\n▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽\n")
    print("요청 문장:", query)
    print(f"🌼{cate} 카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장🌼\n")
    for v, idx in enumerate(top_results[0:5]):
        text = corpus[idx].strip() 
        for i in range(3344):
            if text in df['rv'][i]:
                pname = df['pname'][i]
        print(v+1, pname, '\n','▶',text, "(Score: %.4f)" % (cos_scores[idx]))
    print("\n△△△△△△△△△△△△△△△△△△\n")

        
        
def play_rv():
    query= str(input('검색하기 : '))
    embed, corpus, cate = menu()
    num_return(query, embed, corpus, cate)

In [40]:
play_rv2()

검색하기 : 장미향 수분크림
------------------------------
★원하는 카테고리를 선택해주세요★
1. 전체
2. 기초
3. 색조
------------------------------
→ 1

▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽▽

요청 문장: 장미향 수분크림
▶전체카테고리의 리뷰들 중 가장 유사한 5개 리뷰문장◀

0 에스트라 아토베리어365 크림 기프트 세트(크림 80ml*2+로션 60ml+하이드로에센스 40ml) 
 ▶ 🍀 촉촉한 타입의 수분 에센스 (Score: 0.7792)
1 비욘드 엔젤아쿠아 수분 진정 크림 150ml 1+1 기획 (비건) 
 ▶ 딱 수분을 채워주는 순한 수분크림이기 때문에 (Score: 0.7633)
2 [메이크업아티스트Pick] 에스테덤 셀룰러 워터 미스트 200ml (항산화 미스트) 
 ▶ 에스테덤 셀룰러 워터 미스트는 촉촉함이 잘 유지 되는것 같네요. (Score: 0.7571)
3 [메이크업아티스트Pick] 에스테덤 셀룰러 워터 미스트 200ml (항산화 미스트) 
 ▶ 물미스트인데 촉촉하구 보습력 좋아요 (Score: 0.7491)
4 [단독기획] 토리든 다이브인 저분자 히알루론산 수딩크림 더블기획(100ml+100ml) 
 ▶ 이 제품은 수분크림의 정석 같아요😀 (Score: 0.7448)
